# Nutrition Problem

**Español**

Una nutricionista se encuentra en el proceso de decisión para establecer que cantidad de dos tipos de alimento (A y B) debe incorporar en una dieta sabiendo que el costo por libra de cada uno de ellos es $400 y $300, respectivamente.  Además, se ha establecido que una libra de alimento tipo A contiene 3 miligramos de vitaminas, 6 miligramos de minerales y 4 miligramos de proteínas;  mientras que una libra de alimento tipo B contiene 8 miligramos de vitaminas, 3 miligramos de minerales y 5 miligramos de proteínas. También se debe garantizar consumir como mínimo 240 mg de vitaminas, 120 mg de minerales y 200 mg de proteínas. 

**English**

A nutritionist is in the process of deciding how much of two types of food (A and B) to incorporate into a diet knowing that the cost per pound of each is $400 and $300, respectively.  In addition, it has been established that one pound of type A food contains 3 milligrams of vitamins, 6 milligrams of minerals and 4 milligrams of protein; while one pound of type B food contains 8 milligrams of vitamins, 3 milligrams of minerals and 5 milligrams of protein. It should also be guaranteed to consume at least 240 mg of vitamins, 120 mg of minerals and 200 mg of proteins.

| Foods | Price | Vitamins | Minerals | Protein | 
| ----- | ----- | -------- | -------- | ------- | 
| A | $400 | 3 mg | 6 mg | 4 mg | 
| B | $300 | 8 mg | 3 mg | 5 mg | 
| Minimum amount of nutrients| | 240 mg | 120 mg | 200 mg |

# Problem Statement

## Sets

$F$ = Set of foods

$N$ = Set of Nutrients

## Decision Variables

$x_{f}$ = Number of servings of food $f$

## Parameters

$c_{f}$ = Cost per serving of food $f$,  $\forall f \in F$

$a_{f,n}$ = Amount of nutrients $n$ in food $f$, $\forall f \in F, \forall n \in N$

$Nmin_{n}$ = Minimum amount of nutrients $n$, $\forall n \in N$


## Constraints

1. Limit nutrient consumption 

$$\sum a_{f,n} \cdot x_{f} \geq Nmin_{n}, \forall n \in N$$

2. Consumption lower bound

$$x_{f} \geq 0$$

## Objective

Minimize the total cost of the food

$$\text{min} \sum c_f x_f$$

# Model

In [1]:
import pyomo.environ as pyo

In [2]:
model = pyo.ConcreteModel(name="Nutrition Problem", doc="Nutrition Problem")

In [3]:
# Define sets

FOODS = ["A", "B"]
NUTRIENTS = ["Vitamins", "Minerals", "Protein"]

model.F =pyo.Set(initialize=FOODS, doc="Food Set")
model.N = pyo.Set(initialize=NUTRIENTS, doc="Nutrient Set")

In [4]:
#Define Parameters

cf = {"A": 400, 
    "B": 300}

afn = {
    ("A", "Vitamins"): 3,
    ("A", "Minerals"): 6,
    ("A", "Protein"): 4,
    ("B", "Vitamins"): 8,
    ("B", "Minerals"): 3,
    ("B", "Protein"): 5
}

Nminn = {
    "Vitamins": 240,
    "Minerals": 120,
    "Protein": 200
}

model.cf = pyo.Param(model.F, initialize=cf, doc="Cost of Food")
model.afn = pyo.Param(model.F, model.N , initialize=afn, doc="Amount of Nutrients per Food")
model.Nminn = pyo.Param(model.N, initialize=Nminn, doc="Minimum Amount of Nutrients")

In [5]:
# Define Variables

model.xf = pyo.Var(model.F, domain=pyo.NonNegativeReals, doc="Amount of serving food f")

In [6]:
#Define Constraints

#1. Nutrient constraints

def nutrient_limit_min(model, n):
    return model.Nminn[n] <= sum(model.afn[f,n] * model.xf[f] for f in model.F)

model.nutrient_limit_min = pyo.Constraint(model.N, rule=nutrient_limit_min, doc="Minimum Amount of Nutrients")

In [7]:
#Define Objective

def cost_function(model):
    return sum(model.cf[f] * model.xf[f] for f in model.F)

model.objective = pyo.Objective(rule=cost_function, sense=pyo.minimize, doc="Total Cost of Food per serving")

In [8]:
model.pprint()

Nutrition Problem

    3 Set Declarations
        F : Food Set
            Size=1, Index=None, Ordered=Insertion
            Key  : Dimen : Domain : Size : Members
            None :     1 :    Any :    2 : {'A', 'B'}
        N : Nutrient Set
            Size=1, Index=None, Ordered=Insertion
            Key  : Dimen : Domain : Size : Members
            None :     1 :    Any :    3 : {'Vitamins', 'Minerals', 'Protein'}
        afn_index : Size=1, Index=None, Ordered=True
            Key  : Dimen : Domain : Size : Members
            None :     2 :    F*N :    6 : {('A', 'Vitamins'), ('A', 'Minerals'), ('A', 'Protein'), ('B', 'Vitamins'), ('B', 'Minerals'), ('B', 'Protein')}

    3 Param Declarations
        Nminn : Minimum Amount of Nutrients
            Size=3, Index=N, Domain=Any, Default=None, Mutable=False
            Key      : Value
            Minerals :   120
             Protein :   200
            Vitamins :   240
        afn : Amount of Nutrients per Food
            Size=6,

## Solve the model

In [9]:
solver = pyo.SolverFactory('appsi_highs')

In [10]:
solver.solve(model, tee=True)

Presolving model
3 rows, 2 cols, 6 nonzeros
3 rows, 2 cols, 6 nonzeros
Presolve : Reductions: rows 3(-0); columns 2(-0); elements 6(-0) - Not reduced
Problem not reduced by presolve: solving the LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 3(560) 0s
          2     1.2000000000e+04 Pr: 0(0) 0s
Model   status      : Optimal
Simplex   iterations: 2
Objective value     :  1.2000000000e+04
HiGHS run time      :          0.01


{'Problem': [{'Lower bound': 12000.0, 'Upper bound': 12000.0, 'Number of objectives': 1, 'Number of constraints': 0, 'Number of variables': 0, 'Sense': 1}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Termination message': 'TerminationCondition.optimal'}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [11]:
model.objective.expr()

12000.0

In [12]:
model.xf.display()

xf : Amount of serving food f
    Size=2, Index=F
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      A :     0 :   0.0 :  None : False : False : NonNegativeReals
      B :     0 :  40.0 :  None : False : False : NonNegativeReals


In [13]:
model.nutrient_limit_min.display()

nutrient_limit_min : Size=3
    Key      : Lower : Body  : Upper
    Minerals : 120.0 : 120.0 :  None
     Protein : 200.0 : 200.0 :  None
    Vitamins : 240.0 : 320.0 :  None
